In [1]:
import json
import firebase_admin
from firebase_admin import credentials, firestore

# Inicializar Firebase
cred = credentials.Certificate("goblob-95e2a-6add9b68fd5d.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

# Cargar la taxonomía
with open("/Users/nmlemus/projects/yomap/taxonomy/yomap_taxonomy_last_version.json", "r", encoding="utf-8") as f:
    taxonomy = json.load(f)



In [ ]:
# Subir a Firestore
for cat in taxonomy:
    # Crear documento para la categoría
    cat_ref = db.collection("taxonomy").document()
    cat_data = {
        "name_en": cat["name_en"],
        "name_es": cat["name_es"]
    }
    cat_ref.set(cat_data)

    # Subcolección: subcategories
    for sub in cat["subcategories"]:
        sub_data = {
            "name_en": sub["name_en"],
            "name_es": sub["name_es"]
        }
        cat_ref.collection("subcategories").add(sub_data)


In [2]:
# Subir a una colección plana: service_subcategories
for cat in taxonomy:
    category_en = cat["name_en"]
    category_es = cat["name_es"]
    for sub in cat["subcategories"]:
        doc = {
            "name_en": sub["name_en"],
            "name_es": sub["name_es"],
            "category_en": category_en,
            "category_es": category_es
        }
        db.collection("taxonomy_2").add(doc)

# Este es el que use

In [2]:
import re

# ---------- FUNCIONES DE APOYO ----------
def slugify(text):
    return re.sub(r'\s+', '_', text.strip().lower())

def clean_slug(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove symbols
    text = re.sub(r'\s+', '_', text.strip())    # Replace spaces with underscores
    return text

# ---------- SUBE SUBCATEGORÍAS DESNORMALIZADAS ----------
now = firestore.SERVER_TIMESTAMP  # Firebase genera el timestamp en el servidor

for category in taxonomy:
    parent_slug = clean_slug(category["name_en"])

    # Crear documento para la categoría
    doc = {
        "dateCreated": now,
        "dateUpdated": now,
        "isService": True,
        "isSystemCreated": True,
        "parentId": "",
        "parentSlug": "",
        "slug": parent_slug,
        "text": parent_slug,
        "text_es": clean_slug(category["name_es"]),
        "usedBy": 0,
        "weight": 10
    }
    db.collection("taxonomy").add(doc)

    for sub in category["subcategories"]:
        sub_slug = clean_slug(sub["name_en"])
        doc = {
            "dateCreated": now,
            "dateUpdated": now,
            "isService": True,
            "isSystemCreated": True,
            "parentId": "",
            "parentSlug": parent_slug,
            "slug": sub_slug,
            "text": sub_slug,
            "text_es": clean_slug(sub["name_es"]),
            "usedBy": 0,
            "weight": 10
        }

        db.collection("taxonomy").add(doc)